<a href="https://colab.research.google.com/github/morph-dev/tic-tac-toe-ai/blob/colab/TicTacToe_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# GameDefinition

In [1]:
#@title Rules, State, GameEngine
import numpy as np

def isInsideMatrix(i, j, shape):
  return i >= 0 and i < shape[0] and j >= 0 and j < shape[1]

class Rules:
  def __init__(self, width, height, goal, empty_value=0, player_0_value=1, player_1_value=-1):
    self.width = width
    self.height = height
    self.goal = goal
    self.empty_value = empty_value
    self.player_0_value = player_0_value
    self.player_1_value = player_1_value

class State:
  def __init__(self, rules, board, currentPlayer):
      self.rules = rules
      self.board = board
      self.currentPlayer = currentPlayer
      self.__initializeIsEndAndResult() 

  def __initializeIsEndAndResult(self):
    directions = list(zip(
        [0, 1, 1, 1],
        [1, 1, 0, -1]
    ))
    hasEmpty = False
    for i in range(self.board.shape[0]):
      for j in range(self.board.shape[1]):
        target = self.board[i, j]
        if target == self.rules.empty_value:
          hasEmpty = True
          continue
        for di, dj in directions:
          isMatch = True
          for d in range(1, self.rules.goal):
            ni = i + d * di
            nj = j + d * dj
            if not isInsideMatrix(ni, nj, self.board.shape) or self.board[ni, nj] != target:
              isMatch = False
              break
          if isMatch:
            self.isEnd = True
            self.result = target
            return
    
    self.isEnd = not hasEmpty
    if self.isEnd:
      self.result = self.rules.empty_value

  def __key(self):
    return (
        self.currentPlayer,
        ','.join(map(str, self.board.flatten()))
        )
  def __hash__(self):
    return hash(self.__key())
  def __eq__(self, other):
      return isinstance(other, State) and self.__key() == other.__key()

class GameEngine:
  def __init__(self, rules):
    self.rules = rules

  def newGameState(self):
    return State(
        self.rules,
        np.full((self.rules.height, self.rules.width), self.rules.empty_value),
        self.rules.player_0_value)

  def possibleMoves(self, state):
    moves = []
    for i in range(state.board.shape[0]):
      for j in range(state.board.shape[1]):
        if state.board[i, j] == self.rules.empty_value:
          moves.append((i, j))
    return moves

  def makeMove(self, state, move):
    assert not state.isEnd, "Trying to make a imppossible move. Game is finished. Board: %s" % state.board
    assert state.board[move] == self.rules.empty_value, "Trying to make a imppossible move. Board %s, move %s" % (state.board, move)
    newBoard = state.board.copy()
    newBoard[move] = state.currentPlayer
    newPlayer = self.rules.player_0_value if state.currentPlayer == self.rules.player_1_value else self.rules.player_1_value
    return State(self.rules, newBoard, newPlayer)

# Learning

In [7]:
#@title Model
import random

class Model:
  def __init__(self, rules, learning_rate=0.2, exploration_rate=0.3, decay_gamma=0.9):
    self.rules = rules
    self.states_values = {}
    self.learning_rate = learning_rate
    self.exploration_rate = exploration_rate
    self.decay_gamma = decay_gamma # unused

  def chooseMove(self, gameEngine, state, learning_mode=False):
    if learning_mode and random.random() < self.exploration_rate:
      return random.choice(gameEngine.possibleMoves(state))
    
    bestSimilarity = None
    bestMove = None
    for move in gameEngine.possibleMoves(state):
      newState = gameEngine.makeMove(state, move)
      newStatePolicyValue = self.states_values.get(newState, None)
      if newStatePolicyValue is not None:
        similarity = abs(newStatePolicyValue - state.currentPlayer)
        if bestSimilarity is None or similarity < bestSimilarity:
          bestSimilarity = similarity
          bestMove = move

    return bestMove if bestMove is not None else random.choice(gameEngine.possibleMoves(state))

  def learn(self, states, result):
    reward = result
    for state in reversed(states):
      statePolicyValue = self.states_values.get(state, self.rules.empty_value)
      statePolicyValue += self.learning_rate * (reward - statePolicyValue)
      self.states_values[state] = statePolicyValue
      reward = statePolicyValue


In [48]:
#@title Training
def train(gameEngine, model, numberOfGames, printIncrement = 0.1):
  nextPrintRatio = 0.0
  for i in range(numberOfGames):
    if i >= round(nextPrintRatio * numberOfGames):
      print("Step %d out of %d (%.0f%%)" % (
          i, numberOfGames, 100. * i / numberOfGames))
      nextPrintRatio += printIncrement
    
    states = []
    states.append(gameEngine.newGameState())
    while not states[-1].isEnd:
      move = model.chooseMove(gameEngine, states[-1], learning_mode=True)
      states.append(gameEngine.makeMove(states[-1], move))
    model.learn(states, states[-1].result)




# Gameplay

In [22]:
#@title Players

class Player:
  def __init__(self, name):
    self.name = name
  
  def pickMove(self, gameEngine, state):
    pass

class HumanPlayer(Player):
  def __init__(self):
    Player.__init__(self, "Human")
  
  def pickMove(self, gameEngine, state):
    possibleMoves = gameEngine.possibleMoves(state)
    while True:
      inputText = input("Enter move coordinates: ")
      move = tuple(map(int, inputText.split()))
      if move in possibleMoves:
        return move
      print("Impossible move selected!")
      print("Available moves:")
      print(possibleMoves)

class AiPlayer(Player):
  def __init__(self, model):
    Player.__init__(self, "BleepBloop")
    self.model = model
  
  def pickMove(self, gameEngine, state):
    return self.model.chooseMove(gameEngine, state)

In [10]:
#@title printBoard

def printBoard(state):
  n, m =state.board.shape
  rowBreak = "\n" + "+".join(["---"] * m) + "\n"
  stringValue = {
      1: 'X',
      -1: 'O',
      0: ' ',
  }
  cellFunction = lambda v: ' %s ' % stringValue[v]
  rowFunction = lambda row: '|'.join(map(cellFunction , row))
  print(rowBreak.join(map(rowFunction, state.board)))

In [36]:
#@title playGame
from IPython.display import clear_output

def playGame(gameEngine, player1, player2, model_analysis=None):
  state = gameEngine.newGameState()
  currentPlayer = player1
  nextPlayer = player2

  while True:
    # clear_output()
    printBoard(state)
    if model_analysis:
      values = np.zeros(state.board.shape)
      possible_moves = gameEngine.possibleMoves(state)
      for move in possible_moves:
        values[move] = model_analysis.states_values[
            gameEngine.makeMove(state, move)]
      print(values)
    move = currentPlayer.pickMove(gameEngine, state)
    state = gameEngine.makeMove(state, move)
    if state.isEnd:
      break
    currentPlayer, nextPlayer = nextPlayer, currentPlayer

  print("Game over!")
  if state.result == gameEngine.rules.empty_value:
    print("Tie")
  elif state.result == gameEngine.rules.player_0_value:
    print("%s won!" % player1.name)
  else:
    print("%s won!" % player2.name)

# Play

In [50]:
gameEngine = GameEngine(Rules(3,3,3))
model = Model(gameEngine.rules)
train(gameEngine, model, 200000, printIncrement=0.01)

Step 0 out of 200000 (0%)
Step 2000 out of 200000 (1%)
Step 4000 out of 200000 (2%)
Step 6000 out of 200000 (3%)
Step 8000 out of 200000 (4%)
Step 10000 out of 200000 (5%)
Step 12000 out of 200000 (6%)
Step 14000 out of 200000 (7%)
Step 16000 out of 200000 (8%)
Step 18000 out of 200000 (9%)
Step 20000 out of 200000 (10%)
Step 22000 out of 200000 (11%)
Step 24000 out of 200000 (12%)
Step 26000 out of 200000 (13%)
Step 28000 out of 200000 (14%)
Step 30000 out of 200000 (15%)
Step 32000 out of 200000 (16%)
Step 34000 out of 200000 (17%)
Step 36000 out of 200000 (18%)
Step 38000 out of 200000 (19%)
Step 40000 out of 200000 (20%)
Step 42000 out of 200000 (21%)
Step 44000 out of 200000 (22%)
Step 46000 out of 200000 (23%)
Step 48000 out of 200000 (24%)
Step 50000 out of 200000 (25%)
Step 52000 out of 200000 (26%)
Step 54000 out of 200000 (27%)
Step 56000 out of 200000 (28%)
Step 58000 out of 200000 (29%)
Step 60000 out of 200000 (30%)
Step 62000 out of 200000 (31%)
Step 64000 out of 200000 (

In [59]:
#@title Play
player1 = "BleepBloop" #@param ["BleepBloop", "human"]
player2 = "human" #@param ["BleepBloop", "human"]
show_debug = False #@param {type:"boolean"}

human_player = HumanPlayer()
ai_player = AiPlayer(model)
p1 = human_player if player1 == "human" else ai_player
p2 = human_player if player2 == "human" else ai_player
playGame(
    gameEngine,
    p1,
    p2,
    model if show_debug else None)

   |   |   
---+---+---
   |   |   
---+---+---
   |   |   
   |   |   
---+---+---
   | X |   
---+---+---
   |   |   
Enter move coordinates: 0 0
 O |   |   
---+---+---
   | X |   
---+---+---
   |   |   
 O |   |   
---+---+---
   | X | X 
---+---+---
   |   |   
Enter move coordinates: 1 0
 O |   |   
---+---+---
 O | X | X 
---+---+---
   |   |   
 O |   |   
---+---+---
 O | X | X 
---+---+---
 X |   |   
Enter move coordinates: 0 2
 O |   | O 
---+---+---
 O | X | X 
---+---+---
 X |   |   
 O | X | O 
---+---+---
 O | X | X 
---+---+---
 X |   |   
Enter move coordinates: 2 1
 O | X | O 
---+---+---
 O | X | X 
---+---+---
 X | O |   
Game over!
Tie


In [53]:
import pickle

# '/gdrive/My Drive/policy_200000_20210418.pickle'

def savePolicy(model, filename):
  fw = open(filename, 'wb')
  pickle.dump(model.states_values, fw)
  fw.close()

def loadPolicy(self, filename):
  fr = open(filename, 'rb')
  states_values = pickle.load(fr)
  fr.close()
  return states_values